<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/6_simple-model/hj-simple-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tutorial/6_simple-model/hj-simple-model.ipynb

# 간단한 모델 만들어서 제출해보기

간단하게 모델 만들어서 제출해볼 예정

## 환경세팅

In [10]:
%%capture
!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1 # 최신 버전 라이브러리

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
VALIDATION_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed/scenario/validation/"


In [13]:
import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [14]:
import math
import os
import uuid
import time

import numpy as np
import pandas as pd
import tensorflow as tf

from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

import plotly.graph_objs as go
from plotly.subplots import make_subplots



In [15]:
# submission 생성

from waymo_open_dataset.protos import motion_submission_pb2

submission = motion_submission_pb2.MotionChallengeSubmission()

submission.account_name = 'hyocurity@gmail.com'
submission.unique_method_name = 'HJNet'
submission.authors.append('Hyojin Lee')
submission.authors.append('Jaeseung Lee')
submission.affiliation = 'RO:AD'
submission.description = "Validation Set Test"
submission.method_link = 'NA'
submission.submission_type = submission.MOTION_PREDICTION
submission.uses_lidar_data = False

submission

submission_type: MOTION_PREDICTION
account_name: "hyocurity@gmail.com"
unique_method_name: "HJNet"
authors: "Hyojin Lee"
authors: "Jaeseung Lee"
affiliation: "RO:AD"
description: "Validation Set Test"
method_link: "NA"
uses_lidar_data: false

In [16]:
# train, test 용으로 데이터셋 분할

file_paths = tf.io.gfile.glob(VALIDATION_PATH + "*.tfrecord*")
tf.random.shuffle(file_paths)
dataset_size = len(file_paths)

train_ratio = 0.8
test_ratio = 0.2

train_size = int(dataset_size * train_ratio)
test_size = dataset_size - train_size

train_dataset = tf.data.TFRecordDataset(file_paths[:train_size], compression_type='')
test_dataset = tf.data.TFRecordDataset(file_paths[train_size:], compression_type='')

In [17]:
from waymo_open_dataset.protos import scenario_pb2 

dataset = tf.data.TFRecordDataset(file_paths, compression_type='')
for data in dataset :
   scenario = scenario_pb2.Scenario()
   scenario.ParseFromString(data.numpy())
   break

In [18]:
len(scenario.tracks[0].states)

91

## 모델 생성

In [11]:
import tensorflow as tf

class SimpleMotionPredictionModel(tf.keras.Model):
    def __init__(self, num_classes, input_shape):
        super(SimpleMotionPredictionModel, self).__init__()

        # Convolutional Neural Network
        self.conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=input_shape)
        self.conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu')

        # Long Short-Term Memory
        self.lstm1 = tf.keras.layers.LSTM(256, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(256, return_sequences=False)

        # Fully Connected Layer
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(num_classes)


    def call(self, x):
        # Convolutional Neural Network
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # Long Short-Term Memory
        x = tf.keras.layers.Reshape((-1, 256))(x)
        x = self.lstm1(x)
        x = self.lstm2(x)

        # Fully Connected Layer
        x = self.fc1(x)
        x = self.fc2(x)

        return x


## 데이터 파싱

tracks(agent)만 파싱해보려고 한다. 구조는 다음과 같다.

### Structure
- scenario_id (M개)
  - tracks[0]
    - states[0:9] (past) - input
      - center_x
      - center_y
      - ...
      - velocity_y
    - states[10] (current) - input
    - states[11:90] (future) - output(prediction)
  - tracks[1]
  - ...
  - tracks[N]

### Shape
- Input shape
  - (M, 1, N, 11, 9)
  - M : scenario 개수
  - 1 : scenario_id
  - N : tracks(agents) 개수
  - 11 : past + current states
  - 9 : feature (center_x, center_y 등)
- Output shape
  - (M, 1, N, 80, 2)
  - M : scenario 개수
  - 1 : scenario_id
  - N : tracks(agents) 개수
  - 80 : future states
  - 2 : prediction feature(center_x, center_y)



In [27]:
len(scenario.tracks)

27

In [35]:
len(scenario.tracks[0].states)

91

In [28]:
for data in dataset :
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(data.numpy())
  break

In [34]:
scenario.tracks[0].states[0]

center_x: 5564.89306640625
center_y: 2164.099609375
center_z: 101.36456316413546
length: 4.642587661743164
width: 2.0364904403686523
height: 1.5736640691757202
heading: -0.4103038012981415
velocity_x: 5.21484375
velocity_y: -2.294921875
valid: true

In [36]:
def _parse(raw_scenario_data) :
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(raw_scenario_data.numpy())

  return scenario

scenario_data = np.numpy

for data in dataset :
  scenario = _parse(data)
  scenario_data

  break


In [38]:
scenario.scenario_id

'a3cfb7f093ccd29'

## 학습하기

In [15]:
def _parse(raw_scenario_data):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(raw_scenario_data.numpy())

    return scenario

def train() :

  batch_size = 16 #32
  shuffle_buffer_size = 1500 # dataset_size * 10 ~ dataset_size * 100
  num_epochs = 3

  scenario_data = []
  
  for data in train_dataset :
    proto_string = data.numpy()
    proto = scenario_pb2.Scenario()
    proto.ParseFromString(proto_string)
    scenario_data.append(proto)


  scenario_data = scenario_data.batch(batch_size)
  scenario_data = scenario_data.shuffle(buffer_size=shuffle_buffer_size)

  input_shape=()

  # 모델 클래스 생성
  model = SimpleMotionPredictionModel(num_classes=6, input_shape=input_shape)

  # 모델 컴파일
  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae'])

  # 모델 학습
  model.fit(scenario_data,
            epochs=num_epochs,
            validation_data=scenario_data)
  
  return model

In [ ]:
train()

In [12]:
model = train()

Epoch 1/3


ValueError: ignored

In [ ]:
from waymo_open_dataset.protos import scenario_pb2

cnt = 1
for data in dataset :
  if cnt % 10000 == 0:
    print(f"** Scenario {cnt}")
  cnt += 1
  
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(bytearray(data.numpy()))

  if len(scenario.tracks_to_predict) == 0:
    continue

  scenario_preds = motion_submission_pb2.ChallengeScenarioPredictions()
  scenario_preds.scenario_id = scenario.scenario_id

  for ttp in scenario.tracks_to_predict:
    track = scenario.tracks[ttp.track_index]
    prediction = motion_submission_pb2.SingleObjectPrediction()
    prediction.object_id = track.id

    scored_trajectory = motion_submission_pb2.ScoredTrajectory()

    # 예측해야하는 답안. future. 8초간 0.5초씩 총 16개의 (x,y)를 예측
    # 시작 지점은 15번째부터.
    future_states = track.states[15::5]
    for state in future_states:
      scored_trajectory.trajectory.center_x.append(state.center_x)
      scored_trajectory.trajectory.center_y.append(state.center_y)
    prediction.trajectories.append(scored_trajectory)

    scenario_preds.single_predictions.predictions.append(prediction)

  submission.scenario_predictions.append(scenario_preds) # submission에 추가하기

print(f"** Scenario {cnt}")